In [2]:
import numpy as np
import pandas as pd
data = pd.read_csv('data/data.csv',encoding='gb18030')

In [3]:
data.head()

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [4]:
#删除无用列
useless_columns = ['Unnamed: 0','custid','trade_no', 'bank_card_no', 'first_transaction_time', 'id_name','source']
data = data.drop(useless_columns,axis=1)
#删除一行中空值小于60%的行
data.dropna(thresh = 60,axis=0)
#填充 student_featur 空值 ,以 0 填充 
data['student_feature'].fillna(0,inplace=True)
#转换列  reg_preference_for_trad
city_map = {'一线城市':1,'二线城市':2,'三线城市':3,'其他城市':4,'境外':5}
data['reg_preference_for_trad'] = data['reg_preference_for_trad'].map(city_map)


一线城市    3403
三线城市    1064
境外       150
二线城市     131
其他城市       4
Name: reg_preference_for_trad, dtype: int64


In [13]:
#其它空值以 平均值  填充
data.fillna(data.mean(),inplace=True)

In [18]:
#删除 latest_query_time  loans_latest_time 两个object 列
data.drop(['loans_latest_time','latest_query_time'],axis=1,inplace=True)

In [20]:
y = data['status']
X = data.drop('status',axis=1)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=2018)

In [ ]:
#使用随机森林取得每个特征的重要得分
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(X_train,y_train)

In [31]:
feature_dict = dict(zip(data.columns,rfc.feature_importances_))
print(sorted(feature_dict.items(),key=lambda d:d[1],reverse=True))

[('trans_fail_top_count_enum_last_1_month', 0.05346182955240316), ('history_suc_fee', 0.04739123378137743), ('latest_six_month_apply', 0.03637557698357155), ('trans_day_last_12_month', 0.029126013707717023), ('loans_settle_count', 0.025484292182109403), ('latest_one_month_suc', 0.02168245919114242), ('trans_amount_3_month', 0.01750648550578117), ('repayment_capability', 0.016525464710327006), ('consfin_avg_limit', 0.01647513449765205), ('trans_fail_top_count_enum_last_6_month', 0.016399567300551362), ('status', 0.01639742008770985), ('max_cumulative_consume_later_1_month', 0.016287379364622847), ('first_transaction_day', 0.016041953020547544), ('trans_amount_increase_rate_lately', 0.015262333170822423), ('consfin_max_limit', 0.01519615833198517), ('trans_fail_top_count_enum_last_12_month', 0.015193579662633046), ('avg_price_last_12_month', 0.015129288857819645), ('historical_trans_amount', 0.014913784471187798), ('abs', 0.01482925600860082), ('historical_trans_day', 0.01481361274432558

In [114]:
X[['take_amount_in_later_12_month_highest']].max()

take_amount_in_later_12_month_highest    68000
dtype: int64

In [223]:
# 求IV值  对 take_amount_in_later_12_month_highest 列
from matplotlib import pyplot as plt
import seaborn as sns

#查看数据分布
# X.take_amount_in_later_12_month_highest.hist()




In [237]:
def get_iv(X,y,col):
    # 10 等频分箱 
    new_df = X[[col]].merge(y,left_index=True,right_index=True)
    #人为设置bins
    # bins = [0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,20000000]
    # new_df['col_bin'] = pd.cut(new_df['take_amount_in_later_12_month_highest'],bins,right=False)
    #设置等宽分箱  cut 
    # new_df['col_bin'] = pd.cut(new_df['take_amount_in_later_12_month_highest'],10,right=False)
    #设置等频分箱 qcut
    new_df['col_bin'] = pd.qcut(new_df[col],10,duplicates='drop')
    iv_df = new_df.groupby(['col_bin','status'],as_index=True).count().unstack()
    # iv_df.iloc[:,0].values
    total_df = pd.DataFrame({'good':iv_df.iloc[:,0].values,'bad':iv_df.iloc[:,1]})
    #如果是等宽分箱，会有某箱为空值的情况,需要填充；如果DG为0，此时无法计算WOE值，怎么处理？
    #total_df.fillna(0,inplace=True)
    IV = 0
    for i in range(total_df.shape[0]):
        DG = total_df.iloc[i]['good']/total_df['good'].sum()
        DB = total_df.iloc[i]['bad']/total_df['bad'].sum()
        temp_IV = (DG-DB)*np.log(DG/DB)
#         print(DG,DB,temp_IV)
        IV += temp_IV
    
    print('列 {} 的IV值为:{}'.format(col,IV))
    return (col,IV)

In [238]:
iv_low = {}
iv_middle = {}
iv_high = {}
for i in range(40):
    col,IV = get_iv(X,y,X.columns[i])
    if IV >= 0.03 and IV <0.09:
        iv_low[col]=IV
    elif IV >= 0.09 and IV < 0.29:
        iv_middle[col]=IV
    elif IV >=0.29 and IV < 0.49:
        iv_high[col]=IV
        
print('iv_low',iv_low)
print('iv_middle',iv_middle)
print('iv_high',iv_high)
    

列 low_volume_percent 的IV值为:0.0123645389299806
列 middle_volume_percent 的IV值为:0.02663189211391114
列 take_amount_in_later_12_month_highest 的IV值为:0.011814278128922676
列 trans_amount_increase_rate_lately 的IV值为:0.026925509963379207
列 trans_activity_month 的IV值为:0.013547780703655374
列 trans_activity_day 的IV值为:0.0080993252969917
列 transd_mcc 的IV值为:0.011375622534180391
列 trans_days_interval_filter 的IV值为:0.0245583686098881
列 trans_days_interval 的IV值为:0.017956569767230333
列 regional_mobility 的IV值为:0.007093628838756509
列 student_feature 的IV值为:0.0006098701867066476
列 repayment_capability 的IV值为:0.007343226000244455
列 is_high_user 的IV值为:0.0
列 number_of_trans_from_2011 的IV值为:0.0188879172251111
列 historical_trans_amount 的IV值为:0.037591187366284334
列 historical_trans_day 的IV值为:0.02749364645969553
列 rank_trad_1_month 的IV值为:0.1168352681334168
列 trans_amount_3_month 的IV值为:0.04895074362414185
列 avg_consume_less_12_valid_month 的IV值为:0.0048499282778669845
列 abs 的IV值为:0.015202087782684653
列 top_trans_count_last_